In [ ]:
# Now we are going to test and implement more advanced machine learning algorithms

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
import matplotlib.pyplot as plt
import numpy as np
import os, pandas
from sklearn.neural_network import MLPRegressor

Seed = 72993
np.random.seed(Seed)